In [4]:
from __future__ import print_function, division

from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import os
import matplotlib.pyplot as plt
import sys
import numpy as np

# os.makedirs('images')

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([z, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity, classification = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([z, label], [validity, classification])
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        #########     New code:
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Embedding(10, self.latent_dim)(label)
        label_embedding_flat = Flatten()(label_embedding)
        cmd_model = keras.layers.multiply([noise, label_embedding_flat])
        img = model(cmd_model)
        #########     
        return Model([noise, label], img)


    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))

        img = Input(shape=self.img_shape)
        mdl = model(img)

       
        validity = Dense(1, activation="sigmoid")(mdl)
        classification = Dense(10, activation="softmax")(mdl)
        
        return Model(img, [validity, classification])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, Y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        Y_train = Y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            labels = np.random.randint(0, 10, (batch_size, 1))
            
            # Generate a batch of new images
            gen_imgs = self.generator.predict([noise, labels])
            img_labels = Y_train[idx]



            # Train the discriminator            
            img_labelsss = np.zeros((32, 10))
            for i in range(32):
              img_labelsss[i, img_labels[i]] = 1 
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labelsss])


            labelss = np.zeros((32, 10))
            for i in range(32):
              labelss[i, labels[i]] = 1 
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, labelss])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch([noise, labels], [valid, labelss])

            # Plot the progress
            #print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=20000, batch_size=32, sample_interval=800)

Streaming output truncated to the last 5000 lines.
15000 [D loss: 0.717253, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.962664]
15001 [D loss: 0.683830, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.937452]
15002 [D loss: 0.690409, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.894216]
15003 [D loss: 0.671384, acc.: 71.88%, op_acc: 95.31%] [G loss: 0.936617]
15004 [D loss: 0.700062, acc.: 59.38%, op_acc: 96.88%] [G loss: 0.959590]
15005 [D loss: 0.642072, acc.: 65.62%, op_acc: 100.00%] [G loss: 0.962555]
15006 [D loss: 0.688707, acc.: 48.44%, op_acc: 96.88%] [G loss: 1.022325]
15007 [D loss: 0.733826, acc.: 56.25%, op_acc: 93.75%] [G loss: 0.927770]
15008 [D loss: 0.748714, acc.: 50.00%, op_acc: 100.00%] [G loss: 0.919886]
15009 [D loss: 0.645456, acc.: 62.50%, op_acc: 100.00%] [G loss: 0.926743]
15010 [D loss: 0.714123, acc.: 50.00%, op_acc: 96.88%] [G loss: 0.937981]
15011 [D loss: 0.681754, acc.: 60.94%, op_acc: 95.31%] [G loss: 1.032428]
15012 [D loss: 0.697424, acc.: 56.25%, op_acc: 100.00%] [G